In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
load_dotenv()

True

In [7]:
# Fetch credentials and connection info from environment
user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = os.getenv("MYSQL_HOST", "127.0.0.1")
port = int(os.getenv("MYSQL_PORT", "3306"))
database = os.getenv("MYSQL_DATABASE")

# Create SQLAlchemy engine
connection_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_url)

In [8]:
# Load the Updated Data
query = "SELECT * FROM banking_data_updated"

# Read data into DataFrame
with engine.connect() as conn:
  df = pd.read_sql(query, conn)

# Display result
print("Connected to database:", database)

Connected to database: analysisfinancial


In [9]:
df.head()

,Client ID,Name,Age,Location ID,Joined Bank,Banking Contact,Nationality,Occupation,Fee Structure,Loyalty Classification,Estimated Income,Superannuation Savings,Amount of Credit Cards,Credit Card Balance,Bank Loans,Bank Deposits,Checking Accounts,Saving Accounts,Foreign Currency Account,Business Lending,Properties Owned,Risk Weighting,Gender,Investment Advisor,Banking Relationship,Income Lavel
0,IND81288,Raymond Mills,24,34324,2019-05-06,Anthony Torres,American,Safety Technician IV,High,Jade,75384.77,17677.95,1,484.54,776242.92,1485828.64,603617.88,607332.46,12249.96,1134475.30,1,2,Male,Victor Dean,Retail,Low
1,IND65833,Julia Spencer,23,42205,2001-12-10,Jonathan Hawkins,African,Software Consultant,High,Jade,289834.31,17398.92,1,2256.88,1270615.43,641482.79,229521.37,344635.16,61162.31,2000526.10,1,3,Male,Jeremy Porter,Institutional,Mid
2,IND47499,Stephen Murray,27,7314,2010-01-25,Anthony Berry,European,Help Desk Operator,High,Gold,169935.23,42825.90,2,4568.74,1052715.84,1033401.59,652674.69,203054.35,79071.78,548137.58,1,3,Female,Ernest Knight,Private Bank,Mid
3,IND72498,Virginia Garza,40,34594,2019-03-28,Steve Diaz,American,Geologist II,Mid,Silver,356808.11,5473.15,2,4205.00,121195.06,1048157.49,1048157.49,234685.02,57513.65,1148402.29,0,4,Male,Eric Shaw,Commercial,High
4,IND60181,Melissa Sanders,46,41269,2012-07-20,Shawn Long,American,Assistant Professor,Mid,Platinum,130711.68,48077.60,1,3779.49,1048301.95,487782.53,446644.25,128351.45,30012.14,1674412.12,0,3,Female,Kevin Kim,Retail,Mid


In [10]:
df.dtypes

Client ID                           object
Name                                object
Age                                  int64
Location ID                          int64
Joined Bank                 datetime64[ns]
Banking Contact                     object
Nationality                         object
Occupation                          object
Fee Structure                       object
Loyalty Classification              object
Estimated Income                   float64
Superannuation Savings             float64
Amount of Credit Cards               int64
Credit Card Balance                float64
Bank Loans                         float64
Bank Deposits                      float64
Checking Accounts                  float64
Saving Accounts                    float64
Foreign Currency Account           float64
Business Lending                   float64
Properties Owned                     int64
Risk Weighting                       int64
Gender                              object
Investment 

In [12]:
# Calculate the number of days since the joined date
df['Engagement Days'] = (pd.Timestamp.today() - df['Joined Bank']).dt.days
df.head()

,Client ID,Name,Age,Location ID,Joined Bank,Banking Contact,Nationality,Occupation,Fee Structure,Loyalty Classification,Estimated Income,Superannuation Savings,Amount of Credit Cards,Credit Card Balance,Bank Loans,Bank Deposits,Checking Accounts,Saving Accounts,Foreign Currency Account,Business Lending,Properties Owned,Risk Weighting,Gender,Investment Advisor,Banking Relationship,Income Lavel,Engagement Days
0,IND81288,Raymond Mills,24,34324,2019-05-06,Anthony Torres,American,Safety Technician IV,High,Jade,75384.77,17677.95,1,484.54,776242.92,1485828.64,603617.88,607332.46,12249.96,1134475.30,1,2,Male,Victor Dean,Retail,Low,2259
1,IND65833,Julia Spencer,23,42205,2001-12-10,Jonathan Hawkins,African,Software Consultant,High,Jade,289834.31,17398.92,1,2256.88,1270615.43,641482.79,229521.37,344635.16,61162.31,2000526.10,1,3,Male,Jeremy Porter,Institutional,Mid,8615
2,IND47499,Stephen Murray,27,7314,2010-01-25,Anthony Berry,European,Help Desk Operator,High,Gold,169935.23,42825.90,2,4568.74,1052715.84,1033401.59,652674.69,203054.35,79071.78,548137.58,1,3,Female,Ernest Knight,Private Bank,Mid,5647
3,IND72498,Virginia Garza,40,34594,2019-03-28,Steve Diaz,American,Geologist II,Mid,Silver,356808.11,5473.15,2,4205.00,121195.06,1048157.49,1048157.49,234685.02,57513.65,1148402.29,0,4,Male,Eric Shaw,Commercial,High,2298
4,IND60181,Melissa Sanders,46,41269,2012-07-20,Shawn Long,American,Assistant Professor,Mid,Platinum,130711.68,48077.60,1,3779.49,1048301.95,487782.53,446644.25,128351.45,30012.14,1674412.12,0,3,Female,Kevin Kim,Retail,Mid,4740


In [ ]:
# Step 1: Data Preprocessing
# Select relevant features
features = ['Age', 'Estimated Income', 'Bank Loans', 'Credit Card Balance', 'Bank Deposits', 'Saving Accounts', 'Engagement Days', 
            'Loyalty Classification', 'Gender']
target = ['Risk Weighting']

# Create Updated DataFrame
df = pd.concat([df[features], df[target]], axis=1)
df.head()

,Age,Estimated Income,Bank Loans,Credit Card Balance,Bank Deposits,Saving Accounts,Engagement Days,Loyalty Classification,Gender,Risk Weighting
0,24,75384.77,776242.92,484.54,1485828.64,607332.46,2259,Jade,Male,2
1,23,289834.31,1270615.43,2256.88,641482.79,344635.16,8615,Jade,Male,3
2,27,169935.23,1052715.84,4568.74,1033401.59,203054.35,5647,Gold,Female,3
3,40,356808.11,121195.06,4205.00,1048157.49,234685.02,2298,Silver,Male,4
4,46,130711.68,1048301.95,3779.49,487782.53,128351.45,4740,Platinum,Female,3


In [17]:
df.isnull().sum()

Age                       0
Estimated Income          0
Bank Loans                0
Credit Card Balance       0
Bank Deposits             0
Saving Accounts           0
Engagement Days           0
Loyalty Classification    0
Gender                    0
Risk Weighting            0
dtype: int64

In [18]:
df.duplicated().sum()

np.int64(0)